<a href="https://colab.research.google.com/github/jonggu12/yorehalre_project/blob/main/Use_Of_Model_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### accelerate : hugging face 학습루프 가속화 라이브러리
### peft: LoRA, Prefix Tuning, P-Tuing, Prompt Tuning 과 같은 기법들을 쉽게 사용하도록 나온 라이브러리
### bitsandbytes: gpu 에서 모델을 손쉽게 압축할 수 있는 라이브러리
### trl: TRL (Transformer Reinforcement Learning) 은 transfomer언어 모델의 훈련을 위한 풀스택 라이브러리
%pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 라마2 허깅페이스 모델 불러오기
base_model = "heegyu/llama-2-ko-7b-chat"

compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 어댑터 추가
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/colab Notebooks/senior_hc_model")

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=200,
                do_sample=True,
                temperature=0.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                top_k=3,
                # top_p=0.3,
                repetition_penalty = 1.3,
                framework='pt',
                early_stopping=True
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [ ]:
def get_first_sentence(text):
    sentences = text.split('.')
    if sentences:
        return sentences[0] + '.'
    else:
        return ""

def llama2_output(output_text):
    generated_text = output_text[0]['generated_text']
    sep = generated_text.split('### Response:')
    if len(sep) > 1:
        response = sep[1].split('### Instruction')[0].strip()
        return get_first_sentence(response)
    else:
        response = generated_text.strip()
        return get_first_sentence(response)


In [ ]:
prompt = "골다공증의 치료와 예방을 위해 섭취해야 할 영양소의 조합은?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))


<s>[INST] 골다공증의 치료와 예방을 위해 섭취해야 할 영양소의 조합은? [/INST] Ca과 비타민D이다.


In [ ]:
prompt = "비만의 BMI와 허리둘레는 뭐야?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))

<s>[INST] 비만의 BMI와 허리둘레는 뭐야? [/INST] 비만은 BMI가 25 이상일때 속하며 허리둘레는 남성은 86cm이상, 여성은 80cm 이상으로 표준체중보다 살찌거나 짠 체중이 되는거다.


In [ ]:
prompt = "비타민 K 결핍 시 나타나는 증상은?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))

<s>[INST] 비타민 K 결핍 시 나타나는 증상은? [/INST] 혈액응고가 지연된다 
 결석(northampted kidney)으로 돌출한다.


In [ ]:
prompt = "체구성 성분 중 50세 이후 성인기에 증가하는 것은?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))

<s>[INST] 체구성 성분 중 50세 이후 성인기에 증가하는 것은? [/INST] 기초대사율은 점차 감소한다.


In [ ]:
prompt = "뼈의 발달과 유지에 필수이며 자외선 차단제 사용 시 합성이 저해되고 부갑상샘호르몬에 의해 활성화되는 비타민 은?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))

<s>[INST] 뼈의 발달과 유지에 필수이며 자외선 차단제 사용 시 합성이 저해되고 부갑상샘호르몬에 의해 활성화되는 비타민 은? [/INST] 비타민 D이다.


In [ ]:
prompt = "기초대사량을 측정하기 위해 갖추어야 할 조건으로 옳은 것은?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(llama2_output(result))

<s>[INST] 기초대사량을 측정하기 위해 갖추어야 할 조건으로 옳은 것은? [/INST] 식후 12시간 이후 측정해야 합니다.


### LLAMA2 챗봇 연결 테스트 with gradio

https://pub.towardsai.net/how-to-create-llama-2-chatbot-with-gradio-and-hugging-face-in-free-colab-729fe4fb5734

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
# !pip install --upgrade gradio
!pip install -q gradio==3.45.0
!pip install -q gradio --use-deprecated=legacy-resolver
!pip install typing_extensions --upgrade
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
# llama_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=200,
                do_sample=True,
                temperature=0.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                top_k=3,
                # top_p=0.3,
                repetition_penalty = 1.3,
                framework='pt',
                early_stopping=True
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [ ]:
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return f"<s>[INST] {message} [/INST]"

    formatted_message = f"<s>[INST] {history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""
    sequences = pipe(query)

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output
    sentences = response.split('.')[0] + "."

    print("식건강 챗봇:", sentences.strip())
    return sentences.strip()


In [ ]:
import gradio as gr

examples_md = """
### Example Questions:
- 비만의 BMI와 허리둘레는 뭐야?
- 비타민 K 결핍 시 나타나는 증상은?
- 골다공증의 치료와 예방을 위해 섭취해야 할 영양소의 조합은?
"""

def launch_chat():
    with gr.Blocks() as demo:
        gr.Markdown(examples_md)
        gr.ChatInterface(get_llama_response)

    demo.launch(share=True)


launch_chat()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://420f80063eda2c03cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
